In [ ]:
import os
import requests
import funcs_supabase

In [ ]:
def clean_dictionary_values(input_dict):
    """
    This function takes a dictionary as input, and for each string value,
    it removes leading and trailing whitespaces, extra lines, and converts it to lower case.
    """
    # Iterate over all key-value pairs in the dictionary
    for key, value in input_dict.items():
        if isinstance(value, str):  # Check if the value is a string
            # Strip leading/trailing whitespace, replace newlines with space, and convert to lower case
            cleaned_value = ' '.join(value.split()).lower()
            # Update the value in the dictionary
            input_dict[key] = cleaned_value

    return input_dict

# # Example dictionary
# example_dict = {
#     'key1': '   Some TEXT with Extra   Spaces   \n and new lines\n',
#     'key2': '\n\nAnother\nValue With \t Whitespaces\n   and lines. '
# }

# # Clean the dictionary values
# cleaned_dict = clean_dictionary_values(example_dict)
# cleaned_dict


def get_prompt_variables(data):
    """gets the prompt variables for a given list of data, also removes the prompt_variables from the list
    input and output data format is a list of dictionaries:
    [{'name': 'workflow_id', 'value': '5'},

    """
    prompt_variables_dict = {}
    for item in data:
        if 'prompt_variables' in item['name']:
            parts = item['name'].split('[')
            position = int(parts[1].split(']')[0])
            key_or_value = parts[2].split(']')[0]
            if position not in prompt_variables_dict:
                prompt_variables_dict[position] = {}            
            prompt_variables_dict[position][key_or_value] = item['value']

    prompt_variables = [val for key, val in sorted(prompt_variables_dict.items())]
    
    # clean the prompt_variables
    prompt_variables_cleaned = []
    for pv in prompt_variables:
        pv = clean_dictionary_values(pv)
        prompt_variables_cleaned.append(pv)
    
    prompt_variables = prompt_variables_cleaned

    # remove the prompt_variables from the list
    cleaned_list = []  #
    for item in data:
        if 'prompt_variables' not in item['name']:
            cleaned_list.append(item)

    # cleaned_list.append({'name': 'prompt_variables', 'value': prompt_variables})        
    return (prompt_variables, cleaned_list)

# prompt_variables, input_data_cleaned = get_prompt_variables(input_data)
# print(prompt_variables)
# print(input_data_cleaned)


In [ ]:
def name_value_to_dict(list_of_dicts):
    """Converts a list of dicts with 'name'/'key' and 'value' keys to a single dict."""
    # Initialize an empty dictionary to store the results
    combined_dict = {}
    
    # Loop over each dictionary in the list
    for item in list_of_dicts:
        # The key can be under either 'name' or 'key'
        # We use the `get` method to try 'name' first, then 'key' if 'name' doesn't exist
        key = item.get('name') or item.get('key')
        value = item['value']
        
        # Add the key-value pair to the combined dictionary
        combined_dict[key] = value
        
    return combined_dict


In [ ]:
import funcs_jinja
def variables_into_prompts(input_data_cleaned, prompt_variables):

    for item in input_data_cleaned:
        if "prompt" in item:
            # print (item, input_data_cleaned[item])
            template = input_data_cleaned[item]
            input_data_cleaned[item] = funcs_jinja.render_from_template(template, prompt_variables, False)
            # print (input_data_cleaned[item])

    return input_data_cleaned

# print(input_data_cleaned)
# input_data_cleaned_w_vars = variables_into_prompts(input_data_cleaned, prompt_variables)
# print(input_data_cleaned_w_vars)

In [ ]:
def add_to_media_link_table(data):
    # add to media table and get id
    media_id = funcs_supabase.insert_data("media", data)
    return media_id

def upload_media(upload_filepath, file_content, add_to_media_table=True):
    # upload to storage 
    upload = funcs_supabase.upload_file("media", upload_filepath, file_content)
    final_link = f"https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/{upload_filepath}"
    print(final_link)

    if add_to_media_table:
        data_to_send  = {"url": final_link, "type": "image"}
        media_id = add_to_media_link_table(data_to_send)
        print(media_id)
        return media_id

    else: 
        return final_link



In [ ]:
import comfyui
import random
def send_data_get_imgs(rendered_template):
    images_dict = comfyui.get_images(rendered_template)
    image_binaries = comfyui.collect_image_binaries(images_dict)
    outputs_list = []
    # upload_filepath = f"final_media/{project_id}/{timestamp}.mp4"
    for index, binary_data in enumerate(image_binaries):
        # Construct a filename for each image
        random_number = random.randint(1000, 1000000)
        filename = f"{random_number}_{index + 1}.jpg"
        media_complete = upload_media(filename, binary_data)
        outputs_list.append(media_complete)
    return outputs_list
 

In [ ]:
def get_images_2(input_data_original):

    #1 extract prompt variables and remove from input data
    prompt_variables, input_data_cleaned = get_prompt_variables(input_data_original)
    # print(prompt_variables)
    # print(input_data_cleaned)

    #2 convert input data and prompt variables to dicts
    prompt_variables = name_value_to_dict(prompt_variables)
    print(prompt_variables)
    input_data = name_value_to_dict(input_data_cleaned)
    # print(input_data)

    #3 puts the prompt variables into the input data dict
    input_data = variables_into_prompts(input_data, prompt_variables)
    # print(input_data)

    #4 get the workflow template 
    workflow_id = input_data.get('workflow_id')
    workflow_data = funcs_supabase.select_data('sd_workflows', 'id', workflow_id)
    workflow_data = workflow_data[0]
    workflow_template = workflow_data['workflow_template']
    
    #5 replace the variables in the workflow_template with new input data.
    rendered_template = funcs_jinja.render_from_template(workflow_template, input_data, True)
    print(rendered_template)
    print(type(rendered_template))

    # # get the images
    imgs = send_data_get_imgs(rendered_template)
    return imgs
    

    
input_data = [{'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'masterpiece HDR'}, {'name': 'prompt_text', 'value': '{{man}} in {{city}} with {{dog}}'}, {'name': 'prompt_end', 'value': 'sharp focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'bad hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '3688084'}, {'name': 'batch_size', 'value': '3'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name': 'model', 'value': 'protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors'}, {'name': 'prompt_variables[0][key]', 'value': 'man'}, {'name': 'prompt_variables[0][value]', 'value': 'Sexy, blonde, big butt, skimpy outfit'}, {'name': 'prompt_variables[1][key]', 'value': 'city'}, {'name': 'prompt_variables[1][value]', 'value': 'new york city, chrysler building in the background'}, {'name': 'prompt_variables[2][key]', 'value': 'dog'}, {'name': 'prompt_variables[2][value]', 'value': 'Merle cockapoo'}]
get_images_2(input_data)



In [ ]:
def extract_prompt_variables_from_dict(data_dict):
    """
    This function takes a dictionary input and extracts the 'prompt_variables' key-value pairs.
    The 'prompt_variables' are then converted into a dictionary of their own and returned.
    """
    # Initialize an empty dictionary for the prompt variables
    prompt_variables_dict = {}
    
    # Loop through the data_dict to find all prompt variable entries
    for item in data_dict:
        if item.startswith('prompt_variables'):
            # Extract the key and value for the prompt variable
            key = data_dict[item]['key']
            value = data_dict[item]['value']
            # Add to the prompt variables dictionary
            prompt_variables_dict[key] = value

    return prompt_variables_dict

In [ ]:
input_data = [{'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'HELLO masterpiece HDR'}, {'name': 'prompt_text', 'value': '(man, black hair, New York City skyline in the background), standing alone'}, {'name': 'prompt_end', 'value': 'SHARP focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '4956763'}, {'name': 'batch_size', 'value': '5'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name': 'model', 'value': 'protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors'}, {'name': 'prompt_variables', 'value': [{'key': 'man', 'value': 'indian rabi'}, {'key': 'city', 'value': 'London'}, {'key': 'dog', 'value': 'Golden retriever'}]}]
prompt_variables = extract_prompt_variables_from_dict(input_data)

In [ ]:
input_data = [{'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'masterpiece HDR'}, {'name': 'prompt_text', 'value': '{{man}} in {{city}} with {{dog}}'}, {'name': 'prompt_end', 'value': 'sharp focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'bad hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '3688084'}, {'name': 'batch_size', 'value': '3'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name': 'model', 'value': 'protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors'}, {'name': 'prompt_variables[0][key]', 'value': 'man'}, {'name': 'prompt_variables[0][value]', 'value': 'Sexy, blonde, big butt, skimpy outfit'}, {'name': 'prompt_variables[1][key]', 'value': 'city'}, {'name': 'prompt_variables[1][value]', 'value': 'new york city, chrysler building in the background'}, {'name': 'prompt_variables[2][key]', 'value': 'dog'}, {'name': 'prompt_variables[2][value]', 'value': 'Merle cockapoo'}]
input_data_2 = name_value_to_dict(input_data)
print(type(input_data_2))

prompt_variables = extract_prompt_variables_from_dict(input_data_2)
print(prompt_variables)


In [ ]:
comfyui.queue_prompt(input_data)

In [ ]:

# images_dict = comfyui.get_images(json_string)
# image_binaries = comfyui.collect_image_binaries(images_dict)

In [ ]:
def get_value_from_name(data, key_name):
    """
    Retrieves the value for a given key name from a list of dictionaries.
    
    Parameters:
    data (list): A list of dictionaries, each containing 'name' and 'value' pairs.
    key_name (str): The name of the key for which the value is to be retrieved.
    
    Returns:
    str: The value corresponding to the given key name.
    None: If the key name is not found in the data.
    """
    # Iterate through the list of dictionaries
    for item in data:
        # Check if the 'name' key exists and if it matches the 'key_name'
        if 'name' in item and item['name'] == key_name:
            # Return the 'value' associated with the 'name'
            return item['value']
    # If the key_name is not found, return None
    return None

In [ ]:
inputs = workflow_data.get("inputs", [])
print(inputs)

# Initialize an empty dictionary to store the extracted keys and values
workflow_inputs = {}
# Iterate over each dictionary in the list and add the key-value pairs to the extracted_dict
for item in inputs:
    key = item.get('key')
    value = item.get('value')
    if key and value:  # Make sure the key and value are not None
        workflow_inputs[key] = value


print(workflow_inputs) 

In [ ]:
print(input_data_cleaned)
print(prompt_variables)
print(workflow_data)
print(workflow_inputs)

In [ ]:
for item in input_data_cleaned:
    print(item)

In [ ]:
# replace variables in the input data with the project variables.
# replace the variables in the workflow_template with new input data.




In [ ]:

# input form, get images
def get_images(input_data):
    # print(input_data)
    sd_workflow = input_data["workflow_id"]
    workflow_data = funcs_supabase.select_data('sd_workflows', 'id', sd_workflow)
    workflow_data = workflow_data[0]
    print("WORKFLOW DATA")
    print(workflow_data)

    inputs = workflow_data.get("inputs", [])
    print("INPUTS")
    print(inputs)




    # Initialize an empty dictionary to store the extracted keys and values
    inputs_dict = {}
    # Iterate over each dictionary in the list and add the key-value pairs to the extracted_dict
    for item in inputs:
        key = item.get('key')
        value = item.get('value')
        if key and value:  # Make sure the key and value are not None
            inputs_dict[key] = value




    # print("INPUTS DICT")
    # print(inputs_dict)



    inputs_updated = update_inputs(inputs_dict, input_data)

    workflow_template = workflow_data.get("workflow_template")
    # print("WORKFLOW TEMPLATE")
    # print(workflow_template)

    # Render the template with the inputs
    rendered_template = funcs_jinja.render_from_template(workflow_template, inputs_updated)
    print("SENDING TO COMFY:")
    print(rendered_template)


    # random_number = random.randint(1000, 1000000)
    # my_prompt_text = "Close up documentary image of a sexy a young blonde 20 y.o. mongolian 1girl, in a crowded office, nude, sat on an office chair legs spread wide. knees up. smiling"
    # variables_dict = {"seed": random_number, "prompt_text": my_prompt_text}


    # template_rendered = funcs_jinja.render_from_template(template_content, inputs)
    # print(template_rendered)

    images_dict = comfyui.get_images(rendered_template)
    image_binaries = comfyui.collect_image_binaries(images_dict)

    outputs_list = []
    # upload_filepath = f"final_media/{project_id}/{timestamp}.mp4"

    for index, binary_data in enumerate(image_binaries):
        # Construct a filename for each image
        random_number = random.randint(1000, 1000000)

        filename = f"{random_number}_{index + 1}.jpg"

        media_complete = upload_media(filename, binary_data)

        outputs_list.append(media_complete)


    return outputs_list
 




In [13]:
def get_value_for_name(list_of_dicts, key)   :
    for item in list_of_dicts:
        if item['name'] == key:
            return item['value']

In [16]:
def remove_name_for_value (list_of_dicts, key):
    for item in list_of_dicts:
        if item['name'] == key:
            list_of_dicts.remove(item)
            return list_of_dicts
        
key = 'prompt_variables'
removed_list = remove_name_for_value(data, key)
print(removed_list)

[{'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'HELLO masterpiece HDR'}, {'name': 'prompt_text', 'value': '(man, black hair, New York City skyline in the background), standing alone'}, {'name': 'prompt_end', 'value': 'SHARP focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '4956763'}, {'name': 'batch_size', 'value': '5'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name': 'model', 'value': 'protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors'}]


In [ ]:
# Saved Data Format:


input_data = [{'name': 'project_id', 'value': ''}, {'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'HELLO masterpiece HDR'}, {'name': 'prompt_text', 'value': '{{man}} bakes pizza'}, {'name': 'prompt_end', 'value': 'SHARP focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '4956763'}, {'name': 'batch_size', 'value': '5'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name': 'model', 'value': 'protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors'}, {'name': 'prompt_variables[0][key]', 'value': 'man'}, {'name': 'prompt_variables[0][value]', 'value': 'indian rabi'}, {'name': 'prompt_variables[1][key]', 'value': 'city'}, {'name': 'prompt_variables[1][value]', 'value': 'London'}, {'name': 'prompt_variables[2][key]', 'value': 'dog'}, {'name': 'prompt_variables[2][value]', 'value': 'Golden retriever'}]

data = [{'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'HELLO masterpiece HDR'}, {'name': 'prompt_text', 'value': '(man, black hair, New York City skyline in the background), standing alone'}, {'name': 'prompt_end', 'value': 'SHARP focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '4956763'}, {'name': 'batch_size', 'value': '5'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name': 'model', 'value': 'protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors'}, {'name': 'prompt_variables', 'value': [{'key': 'man', 'value': 'indian rabi'}, {'key': 'city', 'value': 'London'}, {'key': 'dog', 'value': 'Golden retriever'}]}]


In [15]:
prompt_variables= get_value_for_name(data, 'prompt_variables')
print(prompt_variables)

[{'key': 'man', 'value': 'indian rabi'}, {'key': 'city', 'value': 'London'}, {'key': 'dog', 'value': 'Golden retriever'}]


In [ ]:
import sd_images
sd_images.get_images_2(data)



In [12]:
get_prompt_variables(data)

IndexError: list index out of range

In [5]:
import sd_images
sd_images.process_scene(301)

2023-11-10 11:53:22,626:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&id=eq.301 "HTTP/1.1 200 OK"
2023-11-10 11:53:22,731:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.38 "HTTP/1.1 200 OK"
2023-11-10 11:53:22,849:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/sd_workflows?select=%2A&id=eq.5 "HTTP/1.1 200 OK"


{'id': 38, 'created_at': '2023-11-10T11:21:52.890954+00:00', 'project_name': 'Jimmy', 'uuid': 'a688b04e-7f96-4255-b285-82785333a5a6', 'status': 'Active', 'project_description': None, 'project_settings': None, 'speech_audio': None, 'subtitles_file': None, 'words_list': None, 'final_media': None, 'sd_settings': [{'name': 'workflow_id', 'value': '5'}, {'name': 'prompt_start', 'value': 'masterpiece HDR'}, {'name': 'prompt_text', 'value': ''}, {'name': 'prompt_end', 'value': 'sharp focus, amazing fine detail, hyper realistic, lifelike, dramatic lighting'}, {'name': 'prompt_neg', 'value': 'hands, text, watermark, nsfw, painting, drawing, sketch, cartoon, anime, manga, render, 3d, watermark, signature, label, long neck'}, {'name': 'seed', 'value': '49567633'}, {'name': 'batch_size', 'value': '3'}, {'name': 'size-width', 'value': '512'}, {'name': 'size-height', 'value': '512'}, {'name': 'cfg_scale', 'value': '7'}, {'name': 'steps', 'value': '30'}, {'name': 'hi_res_fix', 'value': 'on'}, {'name'

2023-11-10 11:54:09,133:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/media/200566_1.jpg "HTTP/1.1 200 OK"


https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/200566_1.jpg


2023-11-10 11:54:09,350:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media "HTTP/1.1 201 Created"


...

https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/629969_2.jpg
{'id': 6937, 'created_at': '2023-11-10T11:54:10.799235+00:00', 'description': None, 'tags': None, 'custom_fields': None, 'url': 'https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/629969_2.jpg', 'type': 'image', 'caption': None, 'old_filename': None, 'bucket': None, 'filename': None, 'media_data': None, 'thumbnail_url': None, 'source': None, 'source_data': None, 'url_remote': None, 'source_type': None, 'source_id': None, 'input_phrase': None, 'terms': None}


2023-11-10 11:54:12,055:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/media/367895_3.jpg "HTTP/1.1 200 OK"
2023-11-10 11:54:12,168:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/media "HTTP/1.1 201 Created"


https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/367895_3.jpg
{'id': 6938, 'created_at': '2023-11-10T11:54:12.155803+00:00', 'description': None, 'tags': None, 'custom_fields': None, 'url': 'https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/367895_3.jpg', 'type': 'image', 'caption': None, 'old_filename': None, 'bucket': None, 'filename': None, 'media_data': None, 'thumbnail_url': None, 'source': None, 'source_data': None, 'url_remote': None, 'source_type': None, 'source_id': None, 'input_phrase': None, 'terms': None}
{'id': 6936, 'created_at': '2023-11-10T11:54:09.342654+00:00', 'description': None, 'tags': None, 'custom_fields': None, 'url': 'https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/200566_1.jpg', 'type': 'image', 'caption': None, 'old_filename': None, 'bucket': None, 'filename': None, 'media_data': None, 'thumbnail_url': None, 'source': None, 'source_data': None, 'url_remote': None, 'source_type': None, 'sour

2023-11-10 11:54:12,582:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-11-10 11:54:12,656:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"
2023-11-10 11:54:12,737:INFO - HTTP Request: POST https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/album_media_link "HTTP/1.1 201 Created"


{'id': 6937, 'created_at': '2023-11-10T11:54:10.799235+00:00', 'description': None, 'tags': None, 'custom_fields': None, 'url': 'https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/629969_2.jpg', 'type': 'image', 'caption': None, 'old_filename': None, 'bucket': None, 'filename': None, 'media_data': None, 'thumbnail_url': None, 'source': None, 'source_data': None, 'url_remote': None, 'source_type': None, 'source_id': None, 'input_phrase': None, 'terms': None}
{'id': 6938, 'created_at': '2023-11-10T11:54:12.155803+00:00', 'description': None, 'tags': None, 'custom_fields': None, 'url': 'https://fpyltvtkpkrkzortucoa.supabase.co/storage/v1/object/public/media/367895_3.jpg', 'type': 'image', 'caption': None, 'old_filename': None, 'bucket': None, 'filename': None, 'media_data': None, 'thumbnail_url': None, 'source': None, 'source_data': None, 'url_remote': None, 'source_type': None, 'source_id': None, 'input_phrase': None, 'terms': None}


In [9]:
import scene_timings
scene_timings.process(38)

2023-11-10 12:08:09,743:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/scenes?select=%2A&project_id=eq.38&status=eq.Active "HTTP/1.1 200 OK"
2023-11-10 12:08:09,851:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.38 "HTTP/1.1 200 OK"
2023-11-10 12:08:10,032:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.38 "HTTP/1.1 200 OK"
2023-11-10 12:08:10,146:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.38 "HTTP/1.1 200 OK"
2023-11-10 12:08:10,244:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.38 "HTTP/1.1 200 OK"
2023-11-10 12:08:10,313:INFO - HTTP Request: GET https://fpyltvtkpkrkzortucoa.supabase.co/rest/v1/video_creator_projects?select=%2A&id=eq.38 "HTTP/1.1 200 OK"
2023-11-10 12:08:10,410:INFO - HTTP R

The number of scenes in the database and the number of scenes in the processed scene timings do not match.
